<a href="https://colab.research.google.com/github/shubhadeep357/ShadowFox/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd

# 1. Load the dataset directly from Keras
# This contains the same data (crime rate, rooms, etc.) as your file
(train_features, train_labels), (test_features, test_labels) = tf.keras.datasets.boston_housing.load_data()

# 2. Normalize the Data (CRITICAL STEP)
# We subtract the average (mean) and divide by the spread (standard deviation).
# This puts all features on the same "playing field".
mean = train_features.mean(axis=0)
std = train_features.std(axis=0)

train_features = (train_features - mean) / std
test_features = (test_features - mean) / std

print("Data loaded and normalized.")
print(f"Training examples: {len(train_features)}")
print(f"Features per house: {train_features.shape[1]}")

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Data loaded and normalized.
Training examples: 404
Features per house: 13


In [2]:
# 3. Build the Model
def build_model():
    model = models.Sequential()

    # Layer 1: 64 neurons, looking for patterns in the inputs
    model.add(layers.Dense(64, activation='relu', input_shape=(train_features.shape[1],)))

    # Layer 2: 64 neurons, combining patterns
    model.add(layers.Dense(64, activation='relu'))

    # Output Layer: 1 single neuron to output the Price.
    # No activation function is used here because we want a raw number.
    model.add(layers.Dense(1))

    # Compile
    # optimizer='rmsprop' is generally very good for regression
    # loss='mse' (Mean Squared Error) helps the AI minimize the dollar error
    # metrics=['mae'] (Mean Absolute Error) tells us how many dollars we are off by
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

model = build_model()
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,121 (20.00 KB)

 Trainable params: 5,121 (20.00 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# 4. Train
print("Starting training...")
history = model.fit(train_features, train_labels,
                    epochs=100,
                    batch_size=16,
                    validation_split=0.2, # Use 20% of data to check ourselves while training
                    verbose=0) # verbose=0 hides the long list of 100 lines

print("Training finished.")

# 5. Check the final error on the test set
loss, mae = model.evaluate(test_features, test_labels, verbose=0)

print(f"\nAverage Error on Test Data: ${mae * 1000:.2f}")


Starting training...
Training finished.

Average Error on Test Data: $2887.75


In [4]:
# 6. Predict on the first 5 test houses
predictions = model.predict(test_features[:5])

print("------------------------------------------------")
print(f"{'Predicted Price':<20} | {'Actual Price':<20} | {'Difference'}")
print("------------------------------------------------")

for i in range(5):
    pred = predictions[i][0]
    actual = test_labels[i]
    diff = abs(pred - actual)
    print(f"${pred*1000:.2f}             | ${actual*1000:.2f}          | ${diff*1000:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
------------------------------------------------
Predicted Price      | Actual Price         | Difference
------------------------------------------------
$9018.84             | $7200.00          | $1818.84
$18616.32             | $18800.00          | $183.68
$20898.79             | $19000.00          | $1898.79
$31437.11             | $27000.00          | $4437.11
$24588.96             | $22200.00          | $2388.96
